In [1]:
import pandas as pd
import numpy as np

In [ ]:
#rob = pd.read_csv('./robo_autos/curados/robados.csv')
#rec = pd.read_csv('./robo_autos/curados/recuperados.csv')

In [ ]:
#rob.columns

In [ ]:
#rec['tramite_fecha'].sort_values()

In [2]:
todo = pd.read_csv('./curados/robos1819_total.csv')

In [ ]:
todo.groupby('tramite_tipo')['tramite_fecha'].count()

In [ ]:
#ver que columnas se pueden dropear (por empezar todo lo que está por código y nombre)
todo.columns
# 'registro_seccional_codigo', 'automotor_tipo_codigo', 'automotor_marca_codigo', 'automotor_modelo_codigo', 'automotor_uso_codigo', 'titular_domicilio_provincia_indec_id', 'titular_pais_nacimiento_indec_id'

In [3]:
#dropear columnas de códigos
todo.drop(['registro_seccional_codigo', 'automotor_tipo_codigo', 'automotor_marca_codigo', 'automotor_modelo_codigo', 
           'automotor_uso_codigo', 'titular_domicilio_provincia_indec_id', 'titular_pais_nacimiento_indec_id'], 
          axis= 1, inplace= True)

In [4]:
todo.groupby('titular_pais_nacimiento')['tramite_tipo'].count().sort_values(ascending= False)[0:11]
# Los paises de nacimiento están repetidos en mayuscula y minúscula. pasar todo a may o min y unificar

titular_pais_nacimiento
Argentina          37903
ARGENTINA          19143
No aplica           3261
No identificado     1385
Paraguay             794
PARAGUAY             439
Bolivia              346
Uruguay              194
BOLIVIA              183
Perú                 180
Italia               153
Name: tramite_tipo, dtype: int64

In [5]:
# pasar todo pais nacimiento a upper para unificar
todo['titular_pais_nacimiento'] = todo['titular_pais_nacimiento'].map(lambda x: x.upper())

In [6]:
# "normalizar" los strings para que las duplicaciones con y sin tildes se unifiquen.
todo['titular_pais_nacimiento'] = todo['titular_pais_nacimiento'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [7]:
# Tratar casos especiales y cambiar (z franca cdo rivadavia y rivera)
todo['titular_pais_nacimiento'] = todo['titular_pais_nacimiento'].replace('Z.F. CDRO.RIVADAVIA(ARG.)', 'ARGENTINA')
todo['titular_pais_nacimiento'] = todo['titular_pais_nacimiento'].replace('Z.FRANCA CDRO. RIVADAVIA (ARG)', 'ARGENTINA')
todo['titular_pais_nacimiento'] = todo['titular_pais_nacimiento'].replace('Z.F. PUNTA ARENAS (CHILE)', 'CHILE')
todo['titular_pais_nacimiento'] = todo['titular_pais_nacimiento'].replace('RIVERA (URUGUAY)', 'URUGUAY')
# Cambiar "NO APLICA" y "NO IDENTIFICADO" por NaN // REEMPLAZAR POR NO IDENTIFICADO?
todo['titular_pais_nacimiento'] = todo['titular_pais_nacimiento'].replace('NO APLICA', np.nan)
todo['titular_pais_nacimiento'] = todo['titular_pais_nacimiento'].replace('NO IDENTIFICADO', np.nan)

In [8]:
#todo['automotor_anio_modelo'].unique()
#todo[todo['automotor_anio_modelo'] == 20].count()
# en 'automotor_anio_modelo' hay 27 registros que figuran como "20" ver que hacer y como tratarlos // YO DROPEARIA
# los tiro
cond_index = todo[todo['automotor_anio_modelo'] == 20].index
todo.drop(cond_index, inplace = True)

In [ ]:
#todo.groupby('titular_pais_nacimiento')['tramite_tipo'].count().sort_values(ascending= False)
todo.columns

In [9]:
# HACER DESPUES DE DUPLICADOS
## Con 'titular_porcentaje_titularidad' hay que tener cuidado, ver si hay registros duplicados por diferentes porcentajes
#    y de ser así unificarlos y generar una columna 'es_titular_unico' 
#todo['titular_porcentaje_titularidad'].unique()
#todo[(todo['titular_porcentaje_titularidad'] == 20) | (todo['titular_porcentaje_titularidad'] == 80)]
# no parecerían estar repetidos. hacer columna 'es dueño unico'
todo['unico_duenio'] = np.where(todo.titular_porcentaje_titularidad == 100, 1, 0)

In [10]:
## 'titular_genero' hay que unificar no identificado y no aplica --> pasa todo a ser No identificado
#todo.groupby('titular_genero')['tramite_tipo'].count().sort_values(ascending= False)
todo['titular_genero'] = todo['titular_genero'].replace('No aplica', 'No identificado')

In [ ]:
## 'automotor_modelo_descripcion' es imposible. CRUZAR CON EL DE PATENTAMIENTOS Y BUSCAR LAS PRINCIPALES 50 POR REGEX
todo['automotor_modelo_descripcion'].nunique()

----------------------

### VOY A DROPEAR AHROA MODELO PARA PROBAR (14/10) ACORDARSE DE VOLVERLA SI NO CAMBIA

In [11]:
todo.drop('automotor_modelo_descripcion', axis= 1, inplace= True)

In [ ]:
todo.groupby('tramite_tipo')['tramite_fecha'].count()

-----------------------

In [ ]:
## 'automotor_tipo_descripcion' es un bardo, capaz conviene volarla o hacer un serio tratamiento de RegEx
todo['automotor_tipo_descripcion'].unique()
todo['automotor_tipo_descripcion'].nunique()

In [ ]:
## 'automotor_marca_descripcion' es enquilombado también pero se puede manejar con un poco de carigno
#todo['automotor_marca_descripcion'].unique()
todo['automotor_marca_descripcion'].nunique()

--------------------------------------

## ver como unificar los recuperados con robados

In [ ]:
todo.loc[todo['tramite_tipo'] == 'COMUNICACIÓN DE RECUPERO']

In [ ]:
# buscar filas duplicadas sin tener en cuenta tipo ni fecha. TRATAR DE ENCONTRAR ROBOS Y RECUPEROS DEL MISMO AUTO
# si los sorteo por fecha_inscripcion_inicial los tengo ordenados. tengo que iterar para agregar columna?

In [12]:
# sortear por fecha inscripción para tener juntos mismos autos, modelo en caso que se repita fecha y fecha trámite para que esté primero robo desps recuepro (si saqué modelo usar marca acá)
#dupli = todo[todo.drop(['tramite_tipo', 'tramite_fecha'],axis=1).duplicated(keep = False)].sort_values(by= ['fecha_inscripcion_inicial', 'automotor_modelo_descripcion', 'tramite_fecha'])
dupli = todo[todo.drop(['tramite_tipo', 'tramite_fecha'],axis=1).duplicated(keep = False)].sort_values(by= ['fecha_inscripcion_inicial', 'automotor_marca_descripcion', 'tramite_fecha'])

### recheckear bien qué hay que sacar a mano porque unificando nombres de paises cambia el número

In [ ]:
# Checkear manualmente las fechas sospechosas
#dupli.loc[dupli.fecha_inscripcion_inicial == '2017-06-28']

In [13]:
# Sacar a mano los "triplicados" y denuncias duplicadas (no importa sacar un nuevo robo de acá pq despues modifico sobre el orig) , NO. TENGO QUE SACAR TRIPLICADO PORQUE ME CAMBIA EL DATAFRAME DE DUPLICADOS
# robado de nuevo
dupli.drop(dupli[(dupli['fecha_inscripcion_inicial'] == '1995-04-07') & (dupli['tramite_fecha'] == '2019-03-19')].index , inplace= True)
# robo triplicado
dupli.drop(dupli[(dupli['fecha_inscripcion_inicial'] == '1998-09-15')].index , inplace= True)
# robo triplicado
dupli.drop(dupli[(dupli['fecha_inscripcion_inicial'] == '2001-05-04') & (dupli['tramite_fecha'] == '2019-04-05')].index , inplace= True)
# robado de nuevo
dupli.drop(dupli[(dupli['fecha_inscripcion_inicial'] == '2006-01-05') & (dupli['tramite_fecha'] == '2018-12-28')].index , inplace= True)
# un robo duplicado
dupli.drop(dupli[(dupli['fecha_inscripcion_inicial'] == '2008-01-02') & (dupli['tramite_fecha'] == '2018-02-22')].index[0], inplace = True)
# recupero duplicado
dupli.drop(dupli[(dupli['fecha_inscripcion_inicial'] == '2012-11-15') & (dupli['tramite_fecha'] == '2019-04-10')].index , inplace= True)
# robado de nuevo
dupli.drop(dupli[(dupli['fecha_inscripcion_inicial'] == '2013-06-05') & (dupli['tramite_fecha'] == '2019-09-05')].index , inplace= True)
# dos denuncias de robo
dupli.drop(dupli[(dupli['fecha_inscripcion_inicial'] == '2016-10-03') & (dupli['tramite_fecha'] == '2018-03-27')].index , inplace= True)
# robado de nuevo
dupli.drop(dupli[(dupli['fecha_inscripcion_inicial'] == '2017-06-28') & (dupli['tramite_fecha'] == '2019-04-12')].index , inplace= True)

#dupli[(dupli['fecha_inscripcion_inicial'] == '2008-01-02') & (dupli['tramite_fecha'] == '2018-02-22')] #drop by index 14771
#dupli.drop(dupli.index[14771]) // dropear el primero de los dos 

In [13]:
#dupli.groupby('tramite_tipo')['tramite_fecha'].count()
# hay robos duplicados, los tengo que dropear!

tramite_tipo
COMUNICACIÓN DE RECUPERO    858
DENUNCIA DE ROBO O HURTO    940
Name: tramite_fecha, dtype: int64

In [ ]:
# generar columna de indices para el df dupli

# Con for loop
#indeces = []
#for i in range(1,(dupli.shape[0] // 2) + 1):
#    indeces.append(i)
#    indeces.append(i)

# con numpy repeat
#rango = np.arange(1, (dupli.shape[0] // 2) + 1)
#np.repeat(rango, 2)

In [14]:
# generar columna de indices para el df dupli
dupli['indice'] = np.repeat(np.arange(1, (dupli.shape[0] // 2) + 1), 2)

In [ ]:
#dupli.head(10)

In [15]:
# reordenar columnas para llevar indice a primero
cols = dupli.columns.tolist()
cols = cols[-1:] + cols[:-1]
dupli = dupli[cols]

In [16]:
# generar df accesorio para poder hacer la columna con el shift
dupli_test = dupli[['indice', 'tramite_tipo', 'tramite_fecha']]
# hacer la columna directamente en el df dupli
dupli['fecha_recup'] = dupli_test.groupby('indice')['tramite_fecha'].shift(-1)

In [17]:
#modificar df original y dupli para poder tener las mismas columnas y reemplazar directamente
dupli.drop('indice', axis = 1, inplace = True)
dupli.drop(dupli[dupli['tramite_tipo'] == 'COMUNICACIÓN DE RECUPERO'].index, inplace= True)
todo['fecha_recup'] = np.nan
#todo['dias_recu'] = np.nan esto no, al pedo

## HACER UNA COLUMNA QUE SE LLAME DE_ROBO PARA PODER DESPUES BORRAR EN EL DF ORIGINAL AQUELLOS RECUPEROS QUE VIENEN DE ROBO Y DEJAR EL RESTO.

In [ ]:
#dupli.head()

In [18]:
#updatear las filas en todo con las fechas de recupero
todo.update(dupli)

In [19]:
# dropear recuperos en df grande
todo.drop(todo[todo['tramite_tipo'] == 'COMUNICACIÓN DE RECUPERO'].index, inplace= True)

In [20]:
# cambiar tipo de columnas a fechas (HACER ESTO SI O SI)
todo['tramite_fecha'] = pd.to_datetime(todo['tramite_fecha'], format="%Y/%m/%d")
todo['fecha_recup'] = pd.to_datetime(todo['fecha_recup'], format="%Y/%m/%d")

In [21]:
# GENERAR LA COLUMNA NUEVA CON LA DIFERENCIA DE DÍAS
todo['dias'] = todo['fecha_recup'] - todo['tramite_fecha']

In [22]:
todo.shape

(61636, 20)

In [23]:
#todo.loc[todo['fecha_inscripcion_inicial'] == '1973-01-22']
todo.dias.describe()

count                         873
mean     121 days 06:09:29.072164
std       98 days 16:20:41.940867
min               0 days 00:00:00
25%              49 days 00:00:00
50%              98 days 00:00:00
75%             167 days 00:00:00
max             525 days 00:00:00
Name: dias, dtype: object

In [24]:
todo.head()

,tramite_tipo,tramite_fecha,fecha_inscripcion_inicial,registro_seccional_descripcion,registro_seccional_provincia,automotor_origen,automotor_anio_modelo,automotor_tipo_descripcion,automotor_marca_descripcion,automotor_uso_descripcion,titular_tipo_persona,titular_domicilio_localidad,titular_domicilio_provincia,titular_genero,titular_anio_nacimiento,titular_pais_nacimiento,titular_porcentaje_titularidad,unico_duenio,fecha_recup,dias
0,DENUNCIA DE ROBO O HURTO,2018-01-17,2000-10-05,ESTEBAN ECHEVERRIA Nº 1,Buenos Aires,Nacional,2000.0,SEDAN,CHEVROLET,Privado,Física,MONTE GRANDE,BUENOS AIRES,Masculino,1981.0,ARGENTINA,100.0,1.0,NaT,NaT
1,DENUNCIA DE ROBO O HURTO,2018-01-03,2007-11-22,LANUS Nº 1,Buenos Aires,Nacional,2007.0,FURGON 600,PEUGEOT,Privado,Física,VALENTIN ALSINA,BUENOS AIRES,Femenino,1990.0,ARGENTINA,100.0,1.0,NaT,NaT
2,DENUNCIA DE ROBO O HURTO,2018-01-12,1995-02-01,MAR DEL PLATA Nº 02,Buenos Aires,Nacional,1995.0,BERLINA 5 PUERTAS,RENAULT,Privado,Física,UNIDAD TURISTICA CHAPADMALAL,BUENOS AIRES,Masculino,1986.0,ARGENTINA,100.0,1.0,NaT,NaT
3,DENUNCIA DE ROBO O HURTO,2018-01-02,1999-09-28,NECOCHEA Nº 1,Buenos Aires,Nacional,1999.0,BERLINA 3 PUERTAS,RENAULT,Privado,Física,NECOCHEA BS.AS.,BUENOS AIRES,No identificado,1964.0,NaN,100.0,1.0,NaT,NaT
4,DENUNCIA DE ROBO O HURTO,2018-01-09,2006-09-07,PILAR Nº 1,Buenos Aires,Nacional,2006.0,FURGON 600,PEUGEOT,Privado,Física,PTE. DERQUI,BUENOS AIRES,Femenino,1961.0,ARGENTINA,100.0,1.0,NaT,NaT


In [25]:
# guardar csv
todo.to_csv('./curados/curados.csv',index=False, encoding='utf-8-sig')

## pruebas, hay que cambiarle antes el nombre a las columnas

In [ ]:
#todo.sort_values('diff')0
#testear = todo.iloc[[26483, 26770, 25782] ,:]
#TESTEAR CON NANs
#testear = todo.iloc[[0, 58823, 9999, 26770, 25782, 40264, 41557] ,:]
testear = todo.copy()

In [ ]:
# cambiar tipo de columnas a fechas (HACER ESTO SI O SI)
testear['tramite_fecha'] = pd.to_datetime(testear['tramite_fecha'], format="%Y/%m/%d")
testear['fecha_recup'] = pd.to_datetime(testear['fecha_recup'], format="%Y/%m/%d")

In [ ]:
# GENERAR LA COLUMNA NUEVA CON LA DIFERENCIA DE DÍAS
testear['dias'] = testear['fecha_recup'] - testear['tramite_fecha']

In [ ]:
testear.loc[testear['fecha_inscripcion_inicial'] == '1973-01-22']

### acomodar todo y agregar columna RECUPERADO, si/no ó 1/0.
### ver en qué formato dejar días.
### exportar csv para laburar

------------------------

# Para checkear fechas triplicadas o erroneas

In [ ]:
#CREAR DF DE PRUEBA SOLO CON FECHA TRÁMITE Y FECHA ID DE LO QUE ESTÁ DUPLICADO (solo fecha inscripcion)
duplicados = todo[todo.drop(['tramite_tipo', 'tramite_fecha'],axis=1).duplicated(keep = False)].sort_values(by= 'fecha_inscripcion_inicial')[['tramite_tipo', 'tramite_fecha', 'fecha_inscripcion_inicial']]

In [ ]:
#CREAR DF DE PRUEBA SOLO CON FECHA TRÁMITE Y FECHA ID DE LO QUE ESTÁ DUPLICADO
duplicados = todo[todo.drop(['tramite_tipo', 'tramite_fecha'],axis=1).duplicated(keep = False)].sort_values(by= ['fecha_inscripcion_inicial', 'automotor_modelo_descripcion', 'tramite_fecha'])[['tramite_tipo', 'tramite_fecha', 'fecha_inscripcion_inicial']]

In [ ]:
#CREAR DF DE PRUEBA SOLO CON FECHA TRÁMITE Y FECHA ID DE LO QUE ESTÁ DUPLICADO (habiendo dropeado modelo, gano 10 samples)
duplicados = todo[todo.drop(['tramite_tipo', 'tramite_fecha'],axis=1).duplicated(keep = False)].sort_values(by= ['fecha_inscripcion_inicial', 'automotor_marca_descripcion', 'tramite_fecha'])[['tramite_tipo', 'tramite_fecha', 'fecha_inscripcion_inicial']]

In [ ]:
duplicados.shape

In [ ]:
duplicados.head()

In [ ]:
#prueba1 de stack overflow (https://stackoverflow.com/questions/52084133/pandas-create-new-column-based-on-duplicates)
newdf=duplicados.assign(Newid=duplicados.groupby('fecha_inscripcion_inicial').cumcount())
newdf.pivot('fecha_inscripcion_inicial','Newid','tramite_fecha')
# esto anda para ver fecha principio y fin, tnego que ver qué onda con los que tienen más de 2 registros. 
# mostrar index en el pivot para poder ubicar el original?

In [ ]:
# with statement para mostrar solo en este caso las 1000 lineas
with pd.option_context("display.max_rows", 1000): 
        display(newdf.pivot('fecha_inscripcion_inicial','Newid','tramite_fecha'))

---------------------------------------

# CHICHITOS APARTE

In [ ]:
# otra opción de stackoverflow (mismo post)
# VER BIEN. CAPAZ ES NECESARIO AGREGAR COLUMNA CON INDICE COMÚN
from collections import defaultdict
from itertools import count

d = defaultdict(count)

i, r = pd.factorize([*zip(duplicados.fecha_inscripcion_inicial, duplicados.)])
j = np.array([next(d[x]) for x in i])

n, m = len(r), j.max() + 1

b = np.empty((n, m), dtype=np.object)
b[i, j] = df.animal

d1 = pd.DataFrame(r.tolist(), columns=['id', 'name'])
d2 = pd.DataFrame(b, columns=['animal', 'more_animal'])
d1.join(d2)

In [ ]:
from collections import defaultdict
from itertools import count

d = defaultdict(count)

i, r = pd.factorize([*zip(duplicados.fecha_inscripcion_inicial, duplicados.tramite_fecha)])

In [ ]:
r